In [1]:
import numpy as np
import pandas as pd
import random

# GA Class

In [2]:
class MancalaGA:
    
    def __init__(self, self_state, opponent_state, gen, popsize, generation, pc, pm):
        self.num_of_gen = gen
        self.pop_size = popsize
        self.num_of_generation = generation
        self.pc = pc
        self.pm = pm
        self.self_state = self_state # list state ai (lubang) *default [7,7,7,7,7,7,7]
        self.opponent_state = opponent_state # list state player (lubang) *default [7,7,7,7,7,7,7]
        
    
    def individu(self):
        return np.array([np.random.uniform(0,100) for i in range(self.num_of_gen)])
    
    def population(self):
        return np.array([self.individu() for i in range(self.pop_size)])
    
    def debug_lubang_representation(self):
        global h1,h2,h3,h4,h5
        populasi = self.population()
        
        
        for idx, lubang in enumerate(self.self_state):
            idx_lubang = idx
            isi_lubang = lubang

            # get value tiap heuristik
            h1 = self.h1(idx_lubang, isi_lubang)
            h2 = self.h2(idx_lubang, isi_lubang)
            h3 = self.h3(idx_lubang, isi_lubang)
            h4 = self.h4(idx_lubang, isi_lubang)
            h5 = self.h5(idx_lubang, isi_lubang)


            print(f"ini lubang ke {idx_lubang} dengan isi isi_lubang {isi_lubang}")
            print(f"nilai h1 adalah {h1}")
            print(f"nilai h2 adalah {h2}")
            print(f"nilai h3 adalah {h3}")
            print(f"nilai h4 adalah {h4}")
            print(f"nilai h5 adalah {h5}")
            print("\n")
                
    
            
    def calc_fitnes(self, individu, heuristic):
        wh = heuristic*individu
        return np.sum(wh[:self.num_of_gen-1]) - wh[-1]
    
    def self_scan(self):
        data = {}
        for idx, lubang in enumerate(self.self_state):
            h1 = self.h1(idx, lubang)
            h2 = self.h2(idx, lubang)
            h3 = self.h3(idx, lubang)
            h4 = self.h4(idx, lubang)
            h5 = self.h5(idx, lubang)
            
            data[idx] = np.array([h1,h2,h3,h4,h5])
        return data
    
        
    
    def h1(self, idx_lubang, isi_lubang):
        '''
        Timbun sebanyak mungkin rock dalam satu lubang. 
        Karena pada akhir permainan, rock pada lubang akan dipindahkan semuanya pada lumbung. Lubang kanan semakin aman.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state)-(idx_lubang+isi_lubang) == 0:
            return 1.0
        return 0.0
    
    def h2(self, idx_lubang, isi_lubang):
        '''
        Pertahankan agar rock pada sisi kita sebanyak mungkin. (versi umum H1)
        '''
        if isi_lubang == 0:
            return 0.0
        
        total_batu_before = np.sum(self.self_state)
        
        # simulasikan perpindahan batu pada lubang
        cp_state = self.self_state[:]
        
            
        # pecah list menjadi kiri dan kanan
        kiri = cp_state[:idx_lubang]
        tengah = [0]
        kanan = cp_state[idx_lubang+1:]
        
        jml_batu = isi_lubang
        new_kanan = []
        for i in kanan:
            
            if jml_batu == 0:
                break
            else:
                new_kanan.append(i+1)
            
            jml_batu -= 1
        
        new_state = kiri+tengah+kanan

        
        # jumlahkan setiap batu untuk kondisi terbaru
        total_batu_after = np.sum(new_state)
        
        return total_batu_after/total_batu_before
    
    def h3(self, idx_lubang, isi_lubang):
        '''
        Mempertahankan gerakan memindah rock sebanyak mungkin
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state) == (idx_lubang+isi_lubang):
            return 1.0
        return 0.0
    
    def h4(self, idx_lubang, isi_lubang):
        '''
        Memaksimalkan jumlah rock pada lumbung. Dengan memilih langkah mencuri
        '''
        target = idx_lubang + isi_lubang
        
        if isi_lubang == 0:
            return 0.0
        elif 0 not in self.self_state:
            return 0.0
        elif target >= len(self.self_state):
            # out of range
            return 0.0
        elif self.self_state[target] == 0:
            # jika target nilainya 0
            # curi
            jml_curi = self.opponent_state[target]
            return 1-(1/jml_curi)
        else:
            return 0.0

        
    def h4_new(self, idx_lubang, isi_lubang):
        
        target = idx_lubang + isi_lubang
        
        if isi_lubang == 0:
            return 0.0
        elif 0 not in self.self_state:
            return 0.0
        elif target > len(self.self_state):
            # out of range
            return 0.0
        elif self.self_state[target] == 0:
            # jika target nilainya 0
            # curi
            jml_curi = self.opponent_state[jml_pindah]
            return 1-(1/jml_curi)
        else:
            return 0.0
        
    
    def h5(self, idx_lubang, isi_lubang):
        '''
        h5 pada excel dihapus diganti dengan h6 diexcel
        yaitu strategi bertahan
        Menjaga score musuh seminimal mungkin. (heuristik dengan mempertimbangkan 2 gerakan musuh kedepan.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if isi_lubang == 0:
            return 0.0
        
        jml_batu_ke_musuh = abs(len(self.self_state) - (idx_lubang+isi_lubang))
        if jml_batu_ke_musuh > 0: # berarti ada batu yang masuk ke lubang musuh
            jml_batu_ke_musuh = len(self.self_state) if jml_batu_ke_musuh > len(self.self_state) else jml_batu_ke_musuh
            return jml_batu_ke_musuh/len(self.self_state)
        else:
            return 0.0
    
    def selection(self, population, verbose=False):
        '''
        menggunakan metode turnamen dari individu yang berada dalam kandidat
        '''
        scan = self.self_scan()
        
        self.kandidat = []
        
        self.rata_fitness = [0,0,0,0,0,0,0,0,0,0]
        for lubang, heuristic in scan.items():
            if verbose:
                print(f"===========================lubang {lubang}======================================")
                print("\n")
                temp = []
                for idx, individu in enumerate(population):
                    data_individu = {
                        'lubang': lubang,
                        'heuristic': heuristic,
                        'individu': {
                            'individuke':idx,
                            'gen':individu
                        },
                        'fitness': self.calc_fitnes(individu, heuristic)
                    }
                    
                    
                    temp.append(data_individu)
                    self.rata_fitness[idx] += data_individu['fitness']
                    print(data_individu)
                    print("\n")
                

                best_individu_each_lubang = sorted(temp, key=lambda p : p.get('fitness'), reverse=True)
                print(f'best individu di lubang {lubang} adalah {best_individu_each_lubang[0]}')
                # append to kandidat
                self.kandidat.append(best_individu_each_lubang[0])
                print(f"================================================================================")
                print("\n")
                
                
                # masukan individu terbaik dari setiap lubang kedalam list kandidat
                
            else:
                pass
            
    def turnamen(self, pop):
        kandidat= []
        result = self.rata_fitness
        while ((len(kandidat)<4)):
            x = random.randint(0,9)
            if(x not in kandidat):
                kandidat.append(x)
            else:
                pass
        if(result[kandidat[0]] > result[kandidat[1]]):
            x = pop[kandidat[0]]
        else:
            x = pop[kandidat[1]]

        if(result[kandidat[2]] > result[kandidat[3]]):
            y = pop[kandidat[2]]
        else:
            y = pop[kandidat[3]]
#         print("========================================================================")
#         print("x = ",x)
#         print("y = ",y)
        return x, y
    
    def crossover(self, individu1, individu2):
        '''
        crossover dengan single point mutation
        
        return new offspring
        '''
        prandom = np.random.uniform(0,1)
        
        if prandom < self.pc:
            # lakukan crossover
            point = random.randint(0,3)
        
            papa = list(individu1[:point])
            mama = list(individu2[point:])
            offspring = papa+mama 
            return np.array(offspring)     
        else:
            '''
            return kan random mama atau papa
            '''
            prob=random.randint(0,1)
            if(prob==0):
                offspr = list(individu1)
                return np.array(offspr)
            else:
                offspr = list(individu2)
                return np.array(offspr)
    
    def mutation(self, individu, num_of_gen):
        scan = self.self_scan()
        '''
        mutasi
        lakukan mutasi hanya untuk 2 gen dalam 1 individu jika propbabiliti individu lebih kecil dari pm
        '''
        pidv = np.random.uniform(0,1)
        print(pidv)
        if pidv < self.pm:
            
            # lakukan pemilihan random gen
            gen_bermutasi = np.random.choice([i for i in range(self.num_of_gen)], num_of_gen)
            #print(gen_bermutasi)
            
            for i in gen_bermutasi:
                
                individu[i] = np.random.uniform(0,100)
            ##################### hitung FV langsung
            heur=ga.self_scan()
            temporary=[]
            for i in range(len(heur)):
                hasil=heur[i]*individu
                temporary.append(hasil)
            res=0
            for j in range(len(temporary)):
                res+= temporary[j][0]+temporary[j][1]+temporary[j][2]+temporary[j][3]-temporary[j][4]
            result=res
            return individu,result
        
        else:
            heur=ga.self_scan()
            temporary=[]
            for i in range(len(heur)):
                hasil=heur[i]*individu
                temporary.append(hasil)
            res=0
            for j in range(len(temporary)):
                res+= temporary[j][0]+temporary[j][1]+temporary[j][2]+temporary[j][3]-temporary[j][4]
            result=res
            return individu,result
        
    def elitism(self,daftar,new_individu,fv_new):
        temp=daftar[0]
        hapus_idx=0
        for i in range(len(daftar)):
            if(daftar[i]<temp):
                temp=daftar[i]
                hapus_idx = i
                print("hapus idx=",hapus_idx)
        if(fv_new>=daftar[hapus_idx]):
            print('populasi yang terganti =',hapus_idx)
            daftar[hapus_idx]=fv_new
            pop[hapus_idx]=new_individu
            return pop,daftar
        else:
            print("generasi ini tidak terganti")
            return pop,daftar
    def moving(self,daftar):
        temp=daftar[0]
        idx_terpilih=0
        for i in range(len(daftar)):
            if(daftar[i]>temp):
                temp=daftar[i]
                idx_terpilih = i
        print("individu terpilih adalah individu ke ",idx_terpilih)
        return idx_terpilih
    def move_lubang(self,daftar):
        temp=daftar[0]
        idx_terpilih=0
        for i in range(len(daftar)):
            if(daftar[i]>temp):
                temp=daftar[i]
                idx_terpilih = i
        print("lubang terpilih adalah lubang ke ",idx_terpilih)
        return idx_terpilih

## RUN

In [3]:
'''
setting param
'''
SELF_STATE = [1,2,2,4,8,5,12]
OPPONENT_STATE = [6,7,8,9,2,4,12]
NUMBER_OF_GEN = 5
NUMBER_OF_POPULATION = 10
NUMBER_OF_GENERATION = 10
PC = 0.8
PM = 0.5
ga = MancalaGA(SELF_STATE, OPPONENT_STATE, NUMBER_OF_GEN, NUMBER_OF_POPULATION, NUMBER_OF_GENERATION, PC, PM)

In [4]:
ga.self_scan()

{0: array([0.        , 0.97058824, 0.        , 0.        , 0.85714286]),
 1: array([0.        , 0.94117647, 0.        , 0.        , 0.57142857]),
 2: array([0.        , 0.94117647, 0.        , 0.        , 0.42857143]),
 3: array([1.        , 0.88235294, 1.        , 0.        , 0.        ]),
 4: array([0.        , 0.76470588, 0.        , 0.        , 0.71428571]),
 5: array([0.        , 0.85294118, 0.        , 0.        , 0.42857143]),
 6: array([0.        , 0.64705882, 0.        , 0.        , 1.        ])}

In [5]:
pop = ga.population()
pop

array([[0.7136598 , 0.38072413, 0.47125965, 0.50540917, 0.44975424],
       [0.92630813, 0.06043458, 0.13800886, 0.65217483, 0.19988715],
       [0.8202549 , 0.25356025, 0.88112344, 0.61186141, 0.40346593],
       [0.96410102, 0.50318114, 0.56033232, 0.37853093, 0.34041605],
       [0.36959227, 0.29013417, 0.73125666, 0.96046072, 0.34034199],
       [0.14503472, 0.17335021, 0.64283427, 0.80875363, 0.08917012],
       [0.7792195 , 0.07640572, 0.81801218, 0.14446305, 0.71093713],
       [0.98720057, 0.81404679, 0.10148576, 0.32234505, 0.34473295],
       [0.92856391, 0.8385941 , 0.84476961, 0.87305734, 0.59570924],
       [0.34247346, 0.21291198, 0.58183073, 0.24455511, 0.31343651]])

In [6]:
for iterasi in range(0,10):
    print("###################### GENERASI KE ",iterasi," #######################")
    ga.selection(pop, verbose=True)
    ga.rata_fitness
    turnamen = ga.turnamen(pop)
    c = ga.crossover(turnamen[0], turnamen[1])
    new_individu,fv_new=ga.mutation(c,2)
    daftar=ga.rata_fitness
    pop,f=ga.elitism(daftar,new_individu,fv_new)

###################### GENERASI KE  0  #######################
===========================lubang 0======================================


{'lubang': 0, 'heuristic': array([0.        , 0.97058824, 0.        , 0.        , 0.85714286]), 'individu': {'individuke': 0, 'gen': array([0.7136598 , 0.38072413, 0.47125965, 0.50540917, 0.44975424])}, 'fitness': -0.015977278554205054}


{'lubang': 0, 'heuristic': array([0.        , 0.97058824, 0.        , 0.        , 0.85714286]), 'individu': {'individuke': 1, 'gen': array([0.92630813, 0.06043458, 0.13800886, 0.65217483, 0.19988715])}, 'fitness': -0.11267475023591036}


{'lubang': 0, 'heuristic': array([0.        , 0.97058824, 0.        , 0.        , 0.85714286]), 'individu': {'individuke': 2, 'gen': array([0.8202549 , 0.25356025, 0.88112344, 0.61186141, 0.40346593])}, 'fitness': -0.09972534861616875}


{'lubang': 0, 'heuristic': array([0.        , 0.97058824, 0.        , 0.        , 0.85714286]), 'individu': {'individuke': 3, 'gen': array([0.9641

{'lubang': 1, 'heuristic': array([0.        , 0.94117647, 0.        , 0.        , 0.57142857]), 'individu': {'individuke': 2, 'gen': array([ 6.67228098, 60.47287629,  0.88112344, 10.09110821,  0.34041605])}, 'fitness': 56.72112481932586}


{'lubang': 1, 'heuristic': array([0.        , 0.94117647, 0.        , 0.        , 0.57142857]), 'individu': {'individuke': 3, 'gen': array([0.96410102, 0.50318114, 0.56033232, 0.37853093, 0.34041605])}, 'fitness': 0.2790587948081297}


{'lubang': 1, 'heuristic': array([0.        , 0.94117647, 0.        , 0.        , 0.57142857]), 'individu': {'individuke': 4, 'gen': array([ 0.98720057,  0.81404679, 40.40417772, 88.33707685,  0.34473295])}, 'fitness': 0.5691714239525343}


{'lubang': 1, 'heuristic': array([0.        , 0.94117647, 0.        , 0.        , 0.57142857]), 'individu': {'individuke': 5, 'gen': array([ 0.8202549 , 60.47287629,  0.88112344,  9.43879041,  0.44975424])}, 'fitness': 56.658645852102744}


{'lubang': 1, 'heuristic': array([0.      

In [7]:
pop



array([[9.64101023e-01, 3.82499092e+01, 8.81123443e-01, 1.00911082e+01,
        3.40416049e-01],
       [9.64101023e-01, 3.82499092e+01, 4.71259647e-01, 1.53838597e-03,
        4.49754242e-01],
       [6.67228098e+00, 6.04728763e+01, 8.81123443e-01, 1.00911082e+01,
        3.40416049e-01],
       [8.20254903e-01, 6.60928699e+00, 8.42431888e+01, 8.83370768e+01,
        3.44732951e-01],
       [9.87200569e-01, 8.14046787e-01, 4.04041777e+01, 8.83370768e+01,
        3.44732951e-01],
       [8.20254903e-01, 6.04728763e+01, 8.81123443e-01, 9.43879041e+00,
        4.49754242e-01],
       [8.20254903e-01, 6.04728763e+01, 8.81123443e-01, 9.56115446e+01,
        3.40416049e-01],
       [9.87200569e-01, 8.14046787e-01, 1.01485759e-01, 3.22345050e-01,
        3.44732951e-01],
       [9.18463141e+01, 6.04728763e+01, 8.81123443e-01, 2.12291260e+01,
        4.49754242e-01],
       [9.64101023e-01, 3.82499092e+01, 4.71259647e-01, 1.53838597e-03,
        4.49754242e-01]])

In [8]:
ga.rata_fitness

[229.98301539701635,
 229.1357988311108,
 369.0289979884423,
 123.34023387006982,
 44.89672720310825,
 362.73961913769165,
 363.17697190825345,
 4.5940352445261725,
 453.7656783616704,
 229.1357988311108]

In [9]:
best_individu=ga.moving(daftar)

individu terpilih adalah individu ke  8


In [10]:
p=pop[best_individu]

In [11]:
sc=ga.self_scan()

In [12]:
sc

{0: array([0.        , 0.97058824, 0.        , 0.        , 0.85714286]),
 1: array([0.        , 0.94117647, 0.        , 0.        , 0.57142857]),
 2: array([0.        , 0.94117647, 0.        , 0.        , 0.42857143]),
 3: array([1.        , 0.88235294, 1.        , 0.        , 0.        ]),
 4: array([0.        , 0.76470588, 0.        , 0.        , 0.71428571]),
 5: array([0.        , 0.85294118, 0.        , 0.        , 0.42857143]),
 6: array([0.        , 0.64705882, 0.        , 0.        , 1.        ])}

In [13]:
list_scan=[]
for i in sc.items():
    list_scan.append(i[1])

In [14]:
np.array(list_scan)

array([[0.        , 0.97058824, 0.        , 0.        , 0.85714286],
       [0.        , 0.94117647, 0.        , 0.        , 0.57142857],
       [0.        , 0.94117647, 0.        , 0.        , 0.42857143],
       [1.        , 0.88235294, 1.        , 0.        , 0.        ],
       [0.        , 0.76470588, 0.        , 0.        , 0.71428571],
       [0.        , 0.85294118, 0.        , 0.        , 0.42857143],
       [0.        , 0.64705882, 0.        , 0.        , 1.        ]])

In [15]:
skalar=np.array(list_scan)*p
skalar

array([[ 0.        , 58.69426228,  0.        ,  0.        ,  0.38550364],
       [ 0.        , 56.91564828,  0.        ,  0.        ,  0.25700242],
       [ 0.        , 56.91564828,  0.        ,  0.        ,  0.19275182],
       [91.84631413, 53.35842026,  0.88112344,  0.        ,  0.        ],
       [ 0.        , 46.24396422,  0.        ,  0.        ,  0.32125303],
       [ 0.        , 51.57980625,  0.        ,  0.        ,  0.19275182],
       [ 0.        , 39.12950819,  0.        ,  0.        ,  0.44975424]])

In [16]:
hsl=[]
for c in range(len(skalar)):
    h=np.sum(skalar[c][0:4])-skalar[c][-1]
    hsl.append(h)

In [17]:
hsl

[58.30875864874305,
 56.658645852102744,
 56.72289645809629,
 146.08585782832142,
 45.922711194344764,
 51.38705443221407,
 38.67975394784806]

In [18]:
ga.move_lubang(hsl)

lubang terpilih adalah lubang ke  3


3

In [19]:
temp=daftar[0]
for i in range(len(daftar)):
    if(daftar[i]<temp):
        temp=daftar[i]
        hapus_idx=i
    else:
        pass
print('temp =',hapus_idx)
daftar[hapus_idx]=fv_new
pop[hapus_idx]=new_individu

temp = 7


In [20]:
daftar

[229.98301539701635,
 229.1357988311108,
 369.0289979884423,
 123.34023387006982,
 44.89672720310825,
 362.73961913769165,
 363.17697190825345,
 453.7656783616704,
 453.7656783616704,
 229.1357988311108]

In [21]:
pop

array([[9.64101023e-01, 3.82499092e+01, 8.81123443e-01, 1.00911082e+01,
        3.40416049e-01],
       [9.64101023e-01, 3.82499092e+01, 4.71259647e-01, 1.53838597e-03,
        4.49754242e-01],
       [6.67228098e+00, 6.04728763e+01, 8.81123443e-01, 1.00911082e+01,
        3.40416049e-01],
       [8.20254903e-01, 6.60928699e+00, 8.42431888e+01, 8.83370768e+01,
        3.44732951e-01],
       [9.87200569e-01, 8.14046787e-01, 4.04041777e+01, 8.83370768e+01,
        3.44732951e-01],
       [8.20254903e-01, 6.04728763e+01, 8.81123443e-01, 9.43879041e+00,
        4.49754242e-01],
       [8.20254903e-01, 6.04728763e+01, 8.81123443e-01, 9.56115446e+01,
        3.40416049e-01],
       [9.18463141e+01, 6.04728763e+01, 8.81123443e-01, 2.12291260e+01,
        4.49754242e-01],
       [9.18463141e+01, 6.04728763e+01, 8.81123443e-01, 2.12291260e+01,
        4.49754242e-01],
       [9.64101023e-01, 3.82499092e+01, 4.71259647e-01, 1.53838597e-03,
        4.49754242e-01]])

In [22]:
heur=ga.self_scan()
temporary=[]
for i in range(len(heur)):
    hasil=heur[i]*new_individu
    temporary.append(hasil)
res=0
for j in range(len(temporary)):
    res+= temporary[j][0]+temporary[j][1]+temporary[j][2]+temporary[j][3]-temporary[j][4]
result=res/5
print(result)

90.75313567233408
